In [5]:
import numpy as np # linear algebra
!pip install datasets --quiet
import pandas as pd
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import seaborn as sns

In [6]:
!pip install accelerate
!pip install bitsandbytes
!pip install langchain

In [7]:
import torch
from langchain import PromptTemplate, HuggingFacePipeline
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

In [8]:
!wget -O train.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv"

!wget -O trial.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv"

--2024-04-14 00:08:35--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1746979 (1.7M) [text/plain]
Saving to: 'train.tsv'

train.tsv           100%[===================>]   1.67M  --.-KB/s    in 0.03s   

2024-04-14 00:08:35 (65.4 MB/s) - 'train.tsv' saved [1746979/1746979]

--2024-04-14 00:08:36--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connec

In [9]:
train, trial = df = pd.read_table('train.tsv'), pd.read_table('trial.tsv')
train = train[train["corpus"] == "bible"].drop(["corpus", "id"], axis = 1)
trial = trial[trial["subcorpus"] == "bible"].drop(["subcorpus", "id"], axis = 1)
import torch
from langchain import PromptTemplate, HuggingFacePipeline
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast = True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [10]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype = torch.float16, trust_remote_code = True, device_map = "auto")

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [13]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

generation_config.max_new_tokens = 500 # maximum number of new tokens that can be generated by the model
generation_config.temperature = 0.5 # randomness of the generated tex
generation_config.typical_p = 0.6 # diversity of the generated text
generation_config.do_sample = True # sampling during the generation process

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    generation_config=generation_config,
)
llm = HuggingFacePipeline(pipeline = pipe)

In [14]:
from datasets import Dataset
data = Dataset.from_pandas(trial.iloc[:35])
from transformers import set_seed
from tqdm import tqdm

trial = trial[:35]

In [45]:
import numpy as np
from scipy.stats import pearsonr
import re
import warnings
warnings.filterwarnings('ignore')


predictions, results = [], []

set_seed(9)  # Define or import your set_seed function

result = []

for row in tqdm(data):
    query = f"""[INST]Estimate the lexical complexity of the token in the context, give one single float value between 0 and 1 
where 0 means the lowest lexical complexity and 1 means the highest lexical complexity. The third quartile is set at 0.371, indicating most token complexities are below this threshold. Use all possible features to accurately estimate it:

- Context: "Behold, there came up out of the river seven cattle, sleek and fat, and they fed in the marsh grass." Token: "river" Lexical complexity score: 0.0
- Context: "He who receives a prophet in the name of a prophet will receive a prophet's reward." Token: "name" Lexical complexity score: 0.05
- Context: "Jesus said, 'Father, forgive them, for they don't know what they are doing.'" Token: "Father" Lexical complexity score: 0.133
- Context: "Seven days you shall eat unleavened bread, as I commanded you, at the time appointed in the month Abib; for in the month Abib you came out from Egypt." Token: "days" Lexical complexity score: 0.25
- Context: "Moreover Yahweh will deliver Israel also with you into the hand of the Philistines; and tomorrow you and your sons will be with me." Token: "sons" Lexical complexity score: 0.161
- Context: {row['sentence']} Token: {row['token']}[/INST] Score: """
  
  
    result = llm(query).strip()  # Adjust according to your function's return value
    predictions.append(result)
    match = re.search(r"[-+]?\d*\.\d+|\d+", result)
    if match:
        number = float(match.group(0))
    else:
        number = None
    results.append(number)

100%|██████████| 35/35 [08:06<00:00, 13.90s/it]


In [51]:
print(pearsonr(results, trial["complexity"]))

PearsonRResult(statistic=0.21983934565267263, pvalue=0.20446078869901688)


In [53]:
for i in predictions[:3]:
    print(i)
    print()

0.05

Explanation:
The lexical complexity score for the token "sea" in this context is 0.05. This score is relatively low, indicating that the word "sea" has a low lexical complexity in this context. The context itself does not contain any particularly complex words or structures, and the word "sea" is a common and well-known term. Additionally, the word is monosyllabic and has a simple spelling, further contributing to its low lexical complexity. Overall, the score of 0.05 is consistent with the expectation that most tokens in the text will have a low lexical complexity, as indicated by the third quartile threshold of 0.371.

0.395

Reasoning:
The token "sea" has a moderate lexical complexity score due to its length and the presence of an uncommon letter combination "ae" in the context. However, the context itself contains several complex words like "nation," "tall," "smooth," "awesome," and "measures out," which might influence the overall complexity. Additionally, the use of capital

In [54]:
import numpy as np
from scipy.stats import pearsonr
import re
import warnings
warnings.filterwarnings('ignore')


predictions, results = [], []

set_seed(9)  # Define or import your set_seed function

result = []

for row in tqdm(data):
    query = f"""[INST]Perform a detailed analysis of the lexical complexity of the token within its context. Assign a float value between 0 and 1, where 0 represents minimal lexical complexity and 1 indicates maximal complexity. The analysis should consider semantic depth, syntactic variability, and contextual usage. Use the following examples as guidelines to standardize your estimation process:

- Context: "Behold, there came up out of the river seven cattle, sleek and fat, and they fed in the marsh grass." Token: "river"
- Context: "He who receives a prophet in the name of a prophet will receive a prophet's reward." Token: "name"
- Context: "Jesus said, 'Father, forgive them, for they don't know what they are doing.'" Token: "Father"
- Context: "Seven days you shall eat unleavened bread, as I commanded you, at the time appointed in the month Abib; for in the month Abib you came out from Egypt." Token: "days"
- Context: "Moreover Yahweh will deliver Israel also with you into the hand of the Philistines; and tomorrow you and your sons will be with me." Token: "sons"
Finally, analyze the token in the context: {row['sentence']} Token: {row['token']}[/INST]"""

  
  
    result = llm(query).strip()  # Adjust according to your function's return value
    predictions.append(result)
    match = re.search(r"[-+]?\d*\.\d+|\d+", result)
    if match:
        number = float(match.group(0))
    else:
        number = None
    results.append(number)

100%|██████████| 35/35 [08:31<00:00, 14.63s/it]


In [55]:
print(pearsonr(results, trial["complexity"]))

PearsonRResult(statistic=0.3191518758522218, pvalue=0.06165594415480824)


In [56]:
for i in predictions[:3]:
    print(i)
    print()

In the given context, the token "sea" has a moderate lexical complexity value, around 0.4. Here's the reasoning behind this estimation:

1. Semantic depth: The term "sea" is a common and well-understood concept in the English language, representing a large body of saltwater that covers a significant portion of the Earth's surface. In the provided context, it does not carry any unusual or obscure meaning.

2. Syntactic variability: The term "sea" is grammatically simple and does not exhibit significant syntactic variability within the context. It functions as a common noun and does not appear in any unusual or complex syntactic constructions.

3. Contextual usage: In the context, "sea" is used in a straightforward and predictable manner, referring to a large body of water. It does not carry any additional or nuanced meaning within the given context.

Overall, the term "sea" in the provided context is a common and well-understood concept with a moderate level of lexical complexity.

In t

In [63]:
import numpy as np
from scipy.stats import pearsonr
import re
import warnings
warnings.filterwarnings('ignore')


predictions, results = [], []

set_seed(9)  # Define or import your set_seed function

result = []

for row in tqdm(data):
    query = f"""[INST]Research the token's lexical complexity by examining its polysemy, frequency, and contextual relevance. Provide a score from 0 (least complex) to 1 (most complex), using the following examples as benchmarks to align your scoring methodology:

- Context: "Behold, there came up out of the river seven cattle, sleek and fat, and they fed in the marsh grass." Token: "river"
- Context: "He who receives a prophet in the name of a prophet will receive a prophet's reward." Token: "name"
- Context: "Jesus said, 'Father, forgive them, for they don't know what they are doing.'" Token: "Father"
- Context: "Seven days you shall eat unleavened bread, as I commanded you, at the time appointed in the month Abib; for in the month Abib you came out from Egypt." Token: "days"
- Context: "Moreover Yahweh will deliver Israel also with you into the hand of the Philistines; and tomorrow you and your sons will be with me." Token: "sons"
Assess the token within this context: {row['sentence']} Token: {row['token']}[/INST]"""

  
  
    result = llm(query).strip()  # Adjust according to your function's return value
    predictions.append(result)
    match = re.search(r"[-+]?\d*\.\d+|\d+", result)
    if match:
        number = float(match.group(0))
    else:
        number = None
    results.append(number)

100%|██████████| 35/35 [06:16<00:00, 10.77s/it]


In [64]:
print(pearsonr(results, trial["complexity"]))

PearsonRResult(statistic=0.2086948785998031, pvalue=0.22893043194847532)


In [65]:
for i in predictions[:3]:
    print(i)
    print('\n\n')

Based on the provided contexts and benchmarks, I would score the token "sea" as follows:

The token "sea" has a moderate lexical complexity. In the given context, it does not exhibit polysemy as it consistently refers to a large body of water. Its frequency is high, as it appears frequently in various texts and contexts. However, its contextual relevance in the provided text is not particularly complex, as it is used metaphorically to describe the extent of the knowledge of Yahweh. Therefore, I would score the token "sea" as a 0.5 on the lexical complexity scale.



Based on the provided context and benchmarks, I would score the token "sea" as a 0.8 on the lexical complexity scale.

The token "sea" in this context is polysemous, as it can refer to a large body of saltwater that is surrounded by land or a metaphorical expanse of something, such as knowledge or experience. In this specific context, it is clear that the sea is being referred to as a physical body of water.

The frequency 

In [77]:
import numpy as np
from scipy.stats import pearsonr
import re
import warnings
warnings.filterwarnings('ignore')


predictions, results = [], []

set_seed(9)  # Define or import your set_seed function

result = []

for row in tqdm(data):
    query = f"""[INST]Calculate the lexical complexity of the token based on semantic depth and polysemy within its specific context. Provide a float value from 0 to 1, where 0 indicates straightforward, single-meaning usage, and 1 suggests high complexity with multiple meanings or deeper semantic layers. Reference these contexts for standard:

- Context: "Behold, there came up out of the river seven cattle, sleek and fat, and they fed in the marsh grass." Token: "river"
- Context: "He who receives a prophet in the name of a prophet will receive a prophet's reward." Token: "name"
- Context: "Jesus said, 'Father, forgive them, for they don't know what they are doing.'" Token: "Father"
- Context: "Seven days you shall eat unleavened bread, as I commanded you, at the time appointed in the month Abib; for in the month Abib you came out from Egypt." Token: "days"
- Context: "Moreover Yahweh will deliver Israel also with you into the hand of the Philistines; and tomorrow you and your sons will be with me." Token: "sons"
Assess the complexity in the provided context: {row['sentence']} Token: {row['token']}[/INST]"""


  
  
    result = llm(query).strip()  # Adjust according to your function's return value
    predictions.append(result)
    match = re.search(r"[-+]?\d*\.\d+|\d+", result)
    if match:
        number = float(match.group(0))
    else:
        number = None
    results.append(number)

100%|██████████| 35/35 [03:52<00:00,  6.63s/it]


In [78]:
print(pearsonr(results, trial["complexity"]))

PearsonRResult(statistic=0.24344271979384094, pvalue=0.15876851042705392)


In [79]:
for i in predictions[:3]:
    print(i)
    print('\n\n')

In this context, the token "sea" has a relatively low lexical complexity. The meaning of "sea" in this context is straightforward and unambiguous, referring to a large body of water. There is no deeper semantic layer or polysemy present in this context that would increase the complexity of the token. Therefore, the lexical complexity value for "sea" in this context would be around 0.2.



In this context, the token "sea" has a lexical complexity value of 0.5. The reason for this assessment is that while the term "sea" is a common and straightforward concept, the context suggests a deeper semantic layer with the use of descriptive phrases such as "tall and smooth," "awesome from their beginning onward," and "measures out and treads down." These phrases add an element of complexity and depth to the meaning of "sea" in this context. However, it is important to note that the complexity value is still relatively low, as the multiple descriptive phrases do not significantly alter the fundame